In [72]:

def read(filename):
    data = []
    with open(filename) as f:
        line = f.readline().strip()
        while line:
            data.append(line)               
            line = f.readline().strip()
    return data


def find_output(lines):
    output = []
    for line in lines:
        if line.startswith('$'):
            break
        else:
            output.append(line)
    return output


def update_cwd(path, cmd):
    _, arg = cmd.split()
    if arg == '..':
        path.pop()
    else:
        path.append(arg)
        
        
def update_tree(dir_tree, path, output):
    
    current = dir_tree
    for d in path:
        if d in current.keys():
            current = current[d]
        else:
            current[d] = {}
            current = current[d]
    
    # add output
    for o in output:
        t, name = o.split()
        if t == 'dir':
            current[name] = {}
        else:
            current[name] = int(t)
    

def parse_terminal(filename):
    
    # read the input file
    data = read(filename)
    
    # list that stores command output
    command_output = []
    
    # keep track of current directory
    cwd = []
    
    # store the directory tree
    tree = {}
    
    for i, line in enumerate(data):
                
        if line.startswith('$'):
                       
            # search next lines for command output
            if not data[i + 1].startswith('$'):
                command_output = find_output(data[i + 1:])
            else:
                command_output = []
                
            # parse the command
            command = line.strip('$').strip()
                        
            if command.startswith('cd'):
                update_cwd(cwd, command)
                
            elif command.startswith('ls'):
                update_tree(tree, cwd, command_output)
            
        else:
            continue
            
    return tree


def calculate_size(file_system):
        
    total = 0
    dir_list = []
    
    for key, value in file_system.items():
        
        if type(value) == dict:
            
            dl, size = calculate_size(value)
            
            dir_list += dl  # append dir list of directories down the tree
            dir_list.append((key, size))  # append size of this key
            
            total += size  # count total
        
        else:
            
            total += value
        
    return dir_list, total

In [80]:
# part 1 example
tree = parse_terminal('example.txt')
dlist, total = calculate_size(tree)
sum([d[1] for d in dlist if d[1] < 100000])

95437

In [79]:
# part 1
tree = parse_terminal('input.txt')
dlist, total = calculate_size(tree)
sum([d[1] for d in dlist if d[1] < 100000])

1908462

In [98]:
# part 2 example
tree = parse_terminal('example.txt')
dlist, total = calculate_size(tree)

space_needed = 3*1e7 - (7*1e7 - total)
options = [d for d in dlist if d[1] > space_needed]

min(options, key=lambda x: x[1])

('d', 24933642)

In [104]:
# part 2 input
tree = parse_terminal('input.txt')
dlist, total = calculate_size(tree)

space_needed = 3*1e7 - (7*1e7 - total)
min([d for d in dlist if d[1] > space_needed], key=lambda x: x[1])

('zmljzwt', 3979145)